In [1]:
import cv2
import numpy as np
from sklearn.datasets import fetch_lfw_pairs
from faceverification import get_default_face_verificator
from faceverification.exceptions import FaceVerificationError
from matplotlib import pyplot as plt

In [2]:
lfw_pairs = fetch_lfw_pairs(subset='test', 
                            color=True, 
                            funneled=False, 
                            data_home='./benchmark/', 
                            slice_=None, 
                            resize=None)

In [3]:
face_verificator = get_default_face_verificator()

In [4]:
results = []
for entry, target in zip(lfw_pairs.pairs, lfw_pairs.target):
    im1 = cv2.cvtColor(entry[0], cv2.COLOR_RGB2BGR)
    im2 = cv2.cvtColor(entry[1], cv2.COLOR_RGB2BGR)
    im1 = (im1 * 255).round().astype(np.uint8)
    im2 = (im2 * 255).round().astype(np.uint8)
    _, im1 = cv2.imencode(".png", im1)
    _, im2 = cv2.imencode(".png", im2)
    im1 = im1.tobytes()
    im2 = im2.tobytes()
    try:
        is_verified, distance = face_verificator.verify(im1, im2)
    except FaceVerificationError:
        continue
    except cv2.error:
        continue
    results.append((is_verified, bool(target)))

/Users/iavdonin/Library/Caches/pypoetry/virtualenvs/faceverification-woOkVGb8-py3.9/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [5]:
results = np.array(results)

In [6]:
pred = results[:, 0]
target = results[:, 1]

In [7]:
ta = (pred[pred] == target[pred]).sum()
tr = (pred[~pred] == target[~pred]).sum()
fa = (pred[pred] != target[pred]).sum()
fr = (pred[~pred] != target[~pred]).sum()

In [8]:
acc = (ta + tr) / (ta + tr + fa + fr)
far = fa / (ta + fa)
frr = fr / (tr + fr)

In [10]:
acc, far, frr

(0.97875, 0.0078125, 0.03365384615384615)